In [1]:
%matplotlib notebook

from numpy import *
import numpy.linalg 
from numpy.random import rand
from matplotlib.pyplot import *


In [2]:
import subprocess
import shlex
import os

This notebook has two parts. 

* **Part 1:** Here, we create artificial data to simulate "observations" at gauges at a final output time $T$. These gauges are described to be observation stations where PM 2.5 concentrations are collected. In selecting these gauges, we used the United States Environmental Protection Agency Air Quality Data of 2020, to choose some cities in California that recorded high PM 2.5 trends. The final output time... 


* **Part 2:** Estimate the value of $S$ by solving a least squares problem.  This will be a linear or nonlinear  problem depending on the source model being solved, that is, a constant step function or Gaussian model, with dimension equal to the the number of gauges.  The function to be optimized involves a call to ForestClaw. 

As a first step, we test that we can run ForestClaw from this notebook.


### Problem setup

This code assumes that we will only observe solution at final time. 

In [3]:
# This is the command that will get called from the command line
## source_model = 1: constant function
## source_model = 2: gaussian function

source_model = 1  

S_true = 20
alpha_true = 0.0001

# Number of parameters to estimate
nparms = 2

#shell_cmd = './latlong --user:source-model={:d} --user:S0={:.2f} --user:alpha={:.4e}'.format

#!pwd

# Keep track of how many times we call ForestClaw
fclaw_counter = 0


## ForestClaw wrapper
<hr style="border-width:4px; border-color:coral"></hr> 

We can wrap ForestClaw in a Python function.   

* Reads data from `fclaw_options.ini`, spatial and temporal resolution

* A ForestClaw run will read data from a gauge file (created below) and report on values at gauges at regular time intervals, specified in ForestClaw options file `fclaw_options.ini'

* This wrapper takes two input parameters $S$ and $\alpha$ that control the source term.  The source term model is also used, but is set as a global variable above. 

* The output are gauge files, that are read in routines below.  The function $G(S,\alpha)$ will call this function and return the value of $q(x,t)$ at the final time, at each of the gauges.  


In [4]:
def run_ForestClaw(S, alpha):
    # Parameters : 
    global fclaw_counter
    shell_cmd = './latlong --user:source-model={:d} --user:S0={:.16f} --user:alpha={:.16e}'.format
    cmd = shell_cmd(source_model, S, alpha)
    arg_list = shlex.split(cmd) 
    output = subprocess.run(arg_list, capture_output=True,text=True)
    
    fclaw_counter += 1
    return output.returncode, output

### Test ForestClaw wrapper


In [5]:
# Test that we can call ForestClaw

errout,output = run_ForestClaw(S_true,alpha_true)

if errout != 0:
    print("latlong : Something bad happened!")
    print(output.stdout)
    exit(0)
else:
    print("Test successful!")        
    
# Reset counter
fclaw_counter = 0
    
print(output.stdout)

Test successful!
[libsc] This is libsc 2.8.1.57-a813
[libsc] CPP                      /opt/local/bin/mpicc -E
[libsc] CPPFLAGS                 
[libsc] CC                       /opt/local/bin/mpicc
[libsc] CFLAGS                    
[libsc] LDFLAGS                  -Wl,-syslibroot,/Library/Developer/CommandLineTools/SDKs/MacOSX11.sdk -Wl,-flat_namespace -Wl,-commons,use_dylibs -L/opt/local/lib
[libsc] LIBS                     /opt/local/lib/libz.dylib;m
[p4est] This is p4est 2.0.94-00da
[p4est] CPP                      /opt/local/bin/mpicc -E
[p4est] CPPFLAGS                 
[p4est] CC                       /opt/local/bin/mpicc
[p4est] CFLAGS                   -O2 -g -Wall -DP4EST_ENABLE_DEBUG=1 -DSC_ENABLE_DEBUG=1 
[p4est] LDFLAGS                  
[p4est] LIBS                     ;/opt/local/lib/libz.dylib;m
[fclaw] This is ForestClaw 0.1.4880-4dae
[fclaw] CPP                      /opt/local/bin/mpicc -E
[fclaw] CPPFLAGS                 
[fclaw] F77                      
[fclaw] FFL

## Create gauge handing files

<hr style="border-width:4px; border-color:coral"></hr>



ForestClaw will read gauge information from a file `gauges.data`, and produce an output gauge file `gauge000<N>.txt` for each gauge. 


### Writing input gauge files

This file should be in the following format :

**Line 1:** Number of gauges to read (**N**)


**Lines 2-N+1:** Gauge data for each gauge

    <gauge_id>  <x_long>  <y_lat> <t0>  <t1>
    
where `gauge_id` is an ID number associated with the gauge (need not be consecutive). The location of the gauge $(x_{long}, y_{lat})$ specifies the location of the gauge in longitude and latitude degrees.  The longitude is assumed to be in $[-180,180]$ and the latitude is in $[-90,90]$.  Gauge values will be logged over the time interval $[t_0, t_1]$.  

A typical `gauges.data` file looks like : 

    3
        0      -120.50        37.00     0.00e+00     1.00e+10
        1      -120.00        37.00     0.00e+00     1.00e+10
        2      -119.50        37.00     0.00e+00     1.00e+10

        

In [ ]:
def write_gauge_input_file(x_long,y_lat,t0=0,t1=1e10):
    m = len(x_long)
    assert len(y_lat) == m, 'Number of gauge coordinates supplied must match'
    
    f = open('gauges.data','w')
    f.write("{:d}\n".format(m_gauges))
    if isscalar(t0):
        t0v = [t0]*m
        t1v = [t1]*m
    else:
        assert len(t0) == m, 'Number t0 coordinates supplied must match'
        assert len(t1) == m, 'Number t1 coordinates supplied must match'
        t0v = t0
        t1v = t1
    for i in range(m_gauges):
        id = i
        f.write("{:5d} {:12.3f} {:12.3f} {:12.2e} {:12.2e}\n".format(id,p_long[i],p_lat[i],t0v[i],t1v[i]))
    f.close()    
    

In [ ]:
m_gauges = 9

# Define gauges here in long/lat coordinates

p_long = array([-119.0187, -119.8947, -120.625, -120.156, -119.375, -120.156, -119.667, -119.375, -120.1088],dtype='float')
p_lat = array([35.3210, 36.1389, 35.125, 34.5, 35.5938, 35.4375, 34.9429, 35.2812, 34.9681],dtype='float')

write_gauge_input_file(p_long,p_lat)

In [ ]:
%%script bash

cat gauges.data

### Reading output gauge files
Write function to read gauge data output.  This file will read temporal data from each gauge and return a vector of values representing values of gauge data at final time. 

In [ ]:
from os.path import exists

# Read gauge files and collect data

def read_gauge_output_file():
    gauge_file = "gauge{:05d}.txt".format
    qvar_data = empty((m_gauges,1))

    errout = 0
    for i in range(m_gauges):
        gfile = gauge_file(i)
        if not exists(gfile):
            errout = 1
            return qvar_data,errout
        
        # Data matrix has three columns : (level, t, q)
        data = loadtxt(gfile)        
        
        # Get q value at final time
        qvar_data[i] = data[-1,2]

    return qvar_data,errout

## Create data

<hr style="border-width:4px; border-color:coral"></hr>

Here we create some data.   This will read the input gauge file created above and create output files `gauge00001.txt`, 
`gauge00002.txt`, ....

Do one run of the code with a $S_{true}$ value.  Then perturb the data to get our observations. 

In [ ]:
errout, output = run_ForestClaw(S_true,alpha_true)
if errout != 0:
    print("Problem running ForestClaw")
    exit(0)
else:
    print("ForestClaw ran successfully.")

# Read gauges files that were created to get "observations"
qdata,errout = read_gauge_output_file()
if errout != 0:
    print("Problem reading gauge files {:s}".format(gfile))
    exit(0)
else:
    print("Read output gauge files successfully.")
    
# TODO : Add a random perturbation to qvar_data vector entries

qvar_observations = qdata + 0.015 *rand(9,1)# + error

print("Success!")
print(qvar_observations)

# Reset ForestClaw counter
fclaw_counter = 0

## Estimate parameters from data

<hr style="border-width:4px; border-color:coral"></hr>

The least squares problem is then to find

\begin{equation}
\overline{\mathbf p} = \mbox{argmin} \Vert \mathbf G(\mathbf p)  - \mathbf d \Vert^2
\end{equation}

where $\mathbf G(\mathbf p)$ is the vector of values from $m$ gauges for parameter choices in vector $\mathbf p$. 

In [ ]:
# Some problem dependent parameters

def G(p):
    S = p[0]
    alpha = p[1]
    print("S = ",S,";  alpha = ",alpha) #remember to comment this so it doesn't print all the time
    errout = run_ForestClaw(S,alpha)
    if errout[0] != 0:
        print("Problem running ForestClaw")
        exit(0)    
        
    # Read output gauge files generated by ForestClaw
    qdata,errout = read_gauge_output_file()
    if errout != 0:
        print("Problem reading gauge files {:s}".format(gfile))
        exit(0)
        
    return qdata

# Function to be minimized. 
def F(p):    
    
    # If optimizing for S and alpha, p = [S,alpha]
    Gp = G(p)
    Fval = Gp - qvar_observations
    Fval.resize(m_gauges)
    
    return Fval

### Define Jacobian

The Jacobian matrix can be computed using the first-order central difference approximation.  Here, we have to select differential $\Delta \mathbf p$ for each parameter.  

In [ ]:
def Jac_constant(p):
    # Compute Jacobian for source_model 1
    I = eye(nparms)
    hv = array([1, 5e-5])    # Appropriate h values for each variable
    jacm = empty((m_gauges,nparms))
    
    # Set first column
    j = 0
    ej = I[:,j:j+1].reshape((nparms,))
    dp = ej*hv[j]
    fp = F(p + dp)
    fm = F(p - dp)
    fcol = (fp - fm)/(2*hv[j])
    fcol.resize((m_gauges,1))
    jacm[:,j:j+1] = fcol
    
    # Second column is zero for source_model = 1, so we don't need to do anything.
    j = 1
    jacm[:,j:j+1] = zeros((m_gauges,1))
    
    return jacm


def jac(p):
    if source_model == 1:
        # Jacobian is constant
        return J0
    else:
        I = eye(nparms)
        hv = array([1, 5e-5])    # Appropriate h values for each variable
        
        # First column is constant
        jacm = J0

        # set second column of Jacobian (depends on alpha)
        j = 1
        ej = I[:,j:j+1].reshape((nparms,))
        dp = ej*hv[j]
        fp = F(p + dp)
        fm = F(p - dp)
        fcol = (fp - fm)/(2*hv[j])
        fcol.resize((m_gauges,1))
        jacm[:,j:j+1] = fcol
    
    return jacm    

## Solve least squares problem

<hr style="border-width:4px; border-color:coral"></hr>

We solve the nonlinear least squares problem using a routine from SciPy. 

In [ ]:
from scipy.optimize import least_squares

# Set constant Jacobian (for first column, or source_model = 1)
p0 = array([S_true,alpha_true])
J0 = Jac_constant(p0)
        
x0 = array([15,1e-4])
S_soln = least_squares(F,x0,jac = jac, method='lm',gtol=1e-6)
display(S_soln)
xsoln = S_soln.x
print("Solution is ",  S_soln.x)

## Display estimated parameters

<hr style="border-width:4px; border-color:coral"></hr>

We now run ForestClaw one more time with estimated parameters.  This will create output files fort.q*

In [ ]:
from IPython.display import Markdown

S_est = xsoln[0]
sstr = Markdown(r"$S$  (estimated) = ")
#display(sstr,S_est)
print("{:>30s} {:12.4f}".format("S (estimated)",S_est))

alpha_est = xsoln[1]
rstr = Markdown(r"$\alpha$  (estimated) = ")
#display(rstr,alpha_est)
print("{:>30s} {:12.4e}".format("alpha (estimated)",alpha_est))

print("{:>30s} {:12d}".format("Number of ForestClaw calls",fclaw_counter))

In [ ]:
# Create output files fort.*

resid = F(xsoln)
print("residual = \n",resid)

In [ ]:
q_estimates = G(xsoln)
print("q_estimates = ")
print("")
print(q_estimates)

In [ ]:
figure(1)
clf

x = arange(1,10)
plot(x, qvar_observations, 'r*', label='obs')
plot(x, q_estimates, 'b*', label='est')
legend()
xlabel("Gauge")
ylabel("q($\mu gm^{-3}$)")
title("PM 2.5 Concentration")
show()

In [ ]:
figure(2)
clf
plot(x, resid, 'r*')
xlabel("Gauge")
ylabel("Residual")
title("Residuals")
show()

In [ ]:
l2_norm = linalg.norm(resid,2)
print("L2_norm = {:6e}".format(l2_norm))

In [ ]:
# Run ForestClaw using estimated parameters
errout,output = run_ForestClaw(S_est,alpha_est)
print(output.stdout)


In [ ]:
print(qvar_observations)

In [ ]:
print(q_estimates)

In [ ]:
# for i in range(0,len(S_true)):


#     errout,output = run_ForestClaw(S_true[i])

#     if errout != 0:
#         print("latlong : Something bad happened!")
#         print(output.stdout)
#         exit(0)
#     else:
#         print("Test successful!")        
    
# # Reset counter
#     fclaw_counter = 0
    
#     print(output.stdout)